In [6]:
from google.colab import drive
# Step 1: Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Define dataset paths
dataset_root = "/content/drive/MyDrive/TharuWorks/Dataset/train"  # Adjust path if necessary
images_dir = os.path.join(dataset_root, "images")  # RGB images
tracked_labels_dir = os.path.join(dataset_root, "tracked_labels")  # Tracked labels
tracked_images_dir = os.path.join(dataset_root, "tracked_images")  # Save visualization
os.makedirs(tracked_images_dir, exist_ok=True)

# Step 1: Process all tracked frames sequentially
prev_tracked_objects = {}

for label_file in sorted(os.listdir(tracked_labels_dir)):
    if not label_file.endswith(".txt"):
        continue

    # Derive corresponding image filename
    image_filename = label_file.replace(".txt", ".jpg")
    image_path = os.path.join(images_dir, image_filename)
    label_path = os.path.join(tracked_labels_dir, label_file)

    # Check if files exist before proceeding
    if not os.path.exists(image_path) or not os.path.exists(label_path):
        print(f"⚠️ Missing file for {label_file}, skipping...")
        continue

    # Load the RGB image
    image = cv2.imread(image_path)

    if image is None:
        print(f"❌ Error loading RGB image: {image_filename}")
        continue

    # Read tracked label file
    with open(label_path, "r") as f:
        lines = f.readlines()

    img_height, img_width, _ = image.shape
    current_tracked_objects = {}

    # **Step 2: Process Each Bounding Box**
    for line in lines:
        parts = line.strip().split()

        # Ignore "closest" keyword by keeping only the first six numerical values
        parts = parts[:6]

        # Validate line contains exactly six values (class_id, x, y, width, height, depth)
        if len(parts) != 6:
            print(f"⚠️ Invalid line in {label_file}: {line.strip()} (Skipping)")
            continue  # Skip invalid entries

        try:
            class_id, x_center, y_center, width, height, depth = map(float, parts)
        except ValueError:
            print(f"⚠️ Non-numeric value found in {label_file}: {line.strip()} (Skipping)")
            continue  # Skip entries with incorrect format

        # Convert YOLO format (normalized) to pixel values
        x_min = int((x_center - width / 2) * img_width)
        y_min = int((y_center - height / 2) * img_height)
        x_max = int((x_center + width / 2) * img_width)
        y_max = int((y_center + height / 2) * img_height)

        # Ensure bounding box is within valid bounds
        x_min, y_min = max(0, x_min), max(0, y_min)
        x_max, y_max = min(img_width - 1, x_max), min(img_height - 1, y_max)

        # Store object info
        current_tracked_objects[(x_min, y_min, x_max, y_max, class_id)] = depth

    # **Step 3: Highlight Tracked Objects**
    for obj, depth in current_tracked_objects.items():
        x_min, y_min, x_max, y_max, class_id = obj

        # Assign different colors for pedestrians and vehicles
        color = (0, 255, 0)  # Default: Green
        if int(class_id) == 22:  # Vehicle
            color = (255, 0, 0)  # Red
        elif int(class_id) == 11:  # Pedestrian
            color = (0, 0, 255)  # Blue

        # **Draw Rectangle Around the Object**
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)

        # **Add Depth Text**
        text = f"Depth: {depth:.2f}m"
        cv2.putText(image, text, (x_min, y_min - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # **Step 4: Save and Display the Annotated Image**
    annotated_image_path = os.path.join(tracked_images_dir, image_filename)
    cv2.imwrite(annotated_image_path, image)
    print(f"✅ Tracked image saved: {annotated_image_path}")

    # Show the image with bounding boxes
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for Matplotlib
    plt.title(f"Tracked Objects on {image_filename}")
    plt.axis("off")
    plt.show()